**Import libraries**

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import os
from google.colab import drive

**Connect dataset from Google Drive**

In [2]:
drive.mount('/content/drive')
path = "drive/MyDrive/CS230_Project/Jenkins_Rstruct_Data"
!ls $path

MessageError: ignored


## Dataset information

In [ ]:
R = sio.loadmat(path+"/JR_2015-12-04_truncated2.mat")["R"][0]
ntrials = len(R)
shape = R[0]['spikeRaster'].todense().shape
fields = R[0].dtype.names

print("There are %d trials in the R-struct" % ntrials)
print("There are %d electrodes with %d millseconds of data \n" % (shape[0], shape[1]))
print("Fields in this dataset include: ")
for field in fields:
    print("-", field)

## Restructure the data

In [ ]:
def seperate_reaches(R, n_reach_trials, num_electrodes, ms_start, ms, reaches, middle_reach={}, shift_middle=True):
    """
    seperate_reaches:
    -----------------
    - R: neural data
    - n_reach_trials: number of trials we care about
    - num_electrodes: number of electrodes we want to account for
    - ms_start: start index of timeseries spikes
    - ms: length of timeseries
    - reaches: dict of possible reach directions (should have length of 8 if shift_middle is True)
    - middle_reach: dict of mappings for shifting middle reaches
    - shift_middle: boolean indicating if we want to shift the middle
    -----------------
    - Seprates data by reach direction
    - Translates middle movements to reaches from previous point
    """
    if shift_middle and len(reaches) == 9:
        raise Exception("shift_middle is true which means len(reaches) should be 8")
    if not shift_middle and len(reaches) == 8:
        raise Exception("shift_middle is false which means len(reaches) should be 9")
    spike_times = np.ndarray(shape=(n_reach_trials, num_electrodes, ms))
    targets = np.ndarray(shape=(n_reach_trials, 1), dtype=int)

    count = {} # verify counts per reach
    per_reach = {} # seperate spikes per reach
    prev = (round(R[0]['target'][0].item(), 2), round(R[0]['target'][1].item(), 2))

    total = 0
    for n in range(n_reach_trials):
        spike_time = R[n]['spikeRaster'].todense()[:, ms_start: ms_start + ms]
        target_x = round(R[n]['target'][0].item(), 2)
        target_y = round(R[n]['target'][1].item(), 2)
        # first check if middle
        if target_x == round(0.0, 2) and target_y == round(0.0, 2) and shift_middle:
            target_x = round(middle_reach_dict[prev[0]], 2)
            target_y = round(middle_reach_dict[prev[1]], 2)
        for d, dir in enumerate(reaches):
            if target_x == round(dir[0], 2) and target_y == round(dir[1], 2):
                prev = (target_x, target_y)
                if d not in count:
                    count[d] = 0
                    per_reach[d] = []
                per_reach[d].append(spike_time)
                count[d] += 1
                # col[d] = 1
                targets[total, :] = d
                spike_times[total, :, :] = spike_time
                total += 1
                break
    return spike_times, targets, count, per_reach

In [ ]:
n_reach_trials = ntrials - 1 # remove first middle reach
num_electrodes = 96
ms = 300
ms_start = 200

xy_sorted = np.array([
             [120.0, 0.0],
             [84.85, 84.85],
             [0.0, 120.0],
             [-84.85,84.85],
             [-120.0, 0],
             [-84.85, -84.85],
             [0.0, -120],
             [84.85,-84.85]])
xy_sorted_middle = np.array([
             [0.0, 0.0],
             [120.0, 0.0],
             [84.85, 84.85],
             [0.0, 120.0],
             [-84.85,84.85],
             [-120.0, 0],
             [-84.85, -84.85],
             [0.0, -120],
             [84.85,-84.85]])
middle_reach_dict = {
    0.0: 0.0,
    -84.85: 84.85,
    84.85: -84.85,
    -120.0: 120.0,
    120.0: -120.0,
}

In [ ]:
spike_times, targets, count, per_reach = seperate_reaches(R, n_reach_trials, num_electrodes, ms_start, ms, xy_sorted, middle_reach_dict, shift_middle=True)

In [ ]:
spike_times.shape, targets.shape, count, len(per_reach) # verify

### Evenly Sample From Each Class

In [ ]:
import sklearn.model_selection as sk

def even_sample(train_size, n_reach_trials, num_electrodes, ms, count, per_reach):
    num_train = 0
    num_test = 0
    for c in count:
        num_train += int(train_size * count[c])
        num_test += int((1 - train_size) * count[c])
    print(num_train, num_test, n_reach_trials - num_train)
    X_train = np.ndarray(shape=(num_train, num_electrodes, ms))
    X_test = np.ndarray(shape=(n_reach_trials - num_train, num_electrodes, ms))

    Y_train = np.ndarray(shape=(num_train, 1), dtype=int)
    Y_test = np.ndarray(shape=(n_reach_trials - num_train, 1), dtype=int)
    train_start = 0
    test_start = 0
    test = 0
    train = 0
    for d in per_reach:
        reach = np.array(per_reach[d])
        target = np.full((reach.shape[0], 1), d)
        reach_x_train, reach_x_test, reach_y_train, reach_y_test = sk.train_test_split(reach, target, train_size=train_size, random_state = 42)
        train += len(reach_y_train)
        test += len(reach_y_test)
        # print(reach_x_train.shape[0], reach_x_test.shape[0])
        X_train[train_start: train_start + reach_x_train.shape[0], :, :] = reach_x_train
        X_test[test_start: test_start + reach_x_test.shape[0], :, :] = reach_x_test
        Y_train[train_start: train_start + reach_y_train.shape[0], :] = reach_y_train
        Y_test[test_start: test_start + reach_y_test.shape[0], :] = reach_y_test
        # print(Y_test)
        train_start += reach_x_train.shape[0]
        test_start += reach_x_test.shape[0]
    print(train ,test)
    return X_train, Y_train, X_test[:test], Y_test[:test]

In [ ]:
raw_x_train, raw_y_train, raw_x_test, raw_y_test = even_sample(.7, n_reach_trials, num_electrodes, ms, count, per_reach)

### Reshape data and apply one-hot

In [ ]:
X_train = tf.reshape(raw_x_train, (raw_x_train.shape[0], raw_x_train.shape[2], raw_x_train.shape[1]))
X_test = tf.reshape(raw_x_test, (raw_x_test.shape[0], raw_x_test.shape[2], raw_x_test.shape[1]))
X_train = tf.cast(X_train, tf.float32)
X_test = tf.cast(X_test, tf.float32)

In [ ]:
Y_train = []
for t in range(raw_y_train.shape[0]):
    col = np.zeros((len(per_reach),))
    col[raw_y_train[t][0]] = 1
    Y_train.append(col)
Y_train = np.array(Y_train)

Y_test = []
for t in range(raw_y_test.shape[0]):
    col = np.zeros((len(per_reach),))
    col[raw_y_test[t][0]] = 1
    Y_test.append(col)
Y_test = np.array(Y_test)

Y_train.shape, Y_test.shape

In [ ]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform
np.random.seed(1)

In [ ]:
def recall(y_true, y_pred):
    y_true = y_true.numpy()
    y_pred = y_pred.numpy()
    y_true = np.ones_like(y_true)
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    all_positives = np.sum(np.round(np.clip(y_true, 0, 1)))

    recall = true_positives / (all_positives + np.epsilon())
    return recall

def precision(y_true, y_pred):
    y_true = y_true.numpy()
    y_pred = y_pred.numpy()
    y_true = np.ones_like(y_true)
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))

    predicted_positives = np.sum(np.round(np.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + np.epsilon())
    return precision

def f1_score(y_true, y_pred):
    y_true = y_true.numpy()
    y_pred = y_pred.numpy()
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2*((p*r)/(p+r+np.epsilon()))

In [ ]:
def Neuro(input_shape, num_catagories):
    """
    LSTM -> LSTM -> Dense -> Softmax
    Arguments:
    input_shape    -- shape of the input, usually (max_len,)
    num_catagories -- number of catagories (8 or 9)
    Returns:
    model -- a model instance in Keras
    """

    # It should be of shape input_shape and dtype 'int32' (as it contains indices, which are integers).
    input = Input(input_shape, dtype='float32')
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # The returned output should be a batch of sequences.
    X = LSTM(32, return_sequences=True)(input)
    # Add dropout with a probability of 0.5
    X = Dropout(.7)(X) 
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # The returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(.7)(X)  
    # Propagate X through a Dense layer with num_catagories units
    X = Dense(num_catagories)(X) 
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    model = Model(input, X)
    
    
    return model

In [ ]:
model = Neuro((ms, num_electrodes), len(per_reach))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs=35, batch_size = 40, shuffle=True, validation_data=(X_test, Y_test))

In [ ]:
loss, acc = model.evaluate(X_test, Y_test)
print()
print("Test accuracy = ", acc)

In [ ]:
preds = model.predict(X_test)
# wrong = X_test[preds != Y_test]
# wrong
total = 0
wrongs = {}
rights = {}
for i in range(len(preds)):
    pred = preds[i]
    test = Y_test[i]
    a = np.argmax(pred)
    b = np.argmax(test)
    if a == b:
        total += 1
        if b not in rights:
            rights[b] = 0
        rights[b] += 1
    else:
        if b not in wrongs:
            wrongs[b] = 0
        wrongs[b] += 1
acc = total / len(preds)

In [ ]:
acc

In [ ]:
wrongs

In [ ]:
for wrong in wrongs:
    print("reach {} right: {} wrong: {} count: {}".format(wrong, rights[wrong], wrongs[wrong], count[wrong]))
sum(wrongs.values()) + sum(rights.values())

In [ ]:
# from tensorflow.keras.metrics import confusion_matrix
conf = tf.compat.v1.confusion_matrix(np.argmax(Y_test, axis=1), np.argmax(preds, axis=1), num_classes=len(per_reach))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
labels = ["Center", "E", "NE", "N", "NW", "W", "SW", "S", "SE"]

disp = ConfusionMatrixDisplay(confusion_matrix=np.array(conf), display_labels=labels)

disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
# confusion matrix
# performance metrics
# graphs of accuracy and loss
# for base + lstm

# for metrics: https://neptune.ai/blog/keras-metrics